# Course Project: Creating a Variation of Fielding Independent Pitching

create a new Fielding Independent Percentage that includes statcast data. Currently the FIP statistic is calculated as $\frac{13 \cdot HR+3 \cdot (BB + HBP)-2 \cdot K}{IP}+FIP constant$

In [ ]:
import pybaseball as pyb
#aggregating season data
pdata = pyb.pitching_stats(2016,2017,ind=0)

In [6]:
import pybaseball as pyb


#retrieve stats used in calculating FIP
#print(list(pdata.columns.values))
#print(pdata['Name'])

#retrieve statcast data to be added to FIP

#create sFIP 


#retrieve ERA
era = pdata['ERA']


#retrieve FIP
fip = pdata['FIP']


150     2.97
94      2.89
98      3.08
50      2.49
139     2.81
165     3.65
247     3.62
196     2.70
183     3.36
82      2.15
351     3.60
188     3.42
262     3.66
294     3.90
305     3.97
345     3.20
192     3.73
265     3.80
72      3.49
403     3.43
37      1.38
195     3.76
216     3.57
210     3.71
119     3.48
370     3.73
102     3.12
320     3.74
257     3.57
267     3.45
       ...  
670     5.92
762     6.15
918    14.48
599     5.62
722     6.61
731     6.61
796     6.99
727     6.71
868     8.06
690     7.91
715     6.15
787     7.45
632     5.74
822     9.85
780     6.32
682     6.12
707     6.22
706     6.86
830     8.67
656     6.10
865     7.14
860     7.74
809     7.20
673     5.94
712     6.00
799     7.39
746     6.42
788     6.89
659     5.61
759     6.52
Name: FIP, Length: 948, dtype: float64
